Manufacturing AI Agent

- Install Dependencies
  
```bash 
pipenv shell
pipenv install langchain langchain-google-genai
```

In [ ]:
import os
import json
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.messages import HumanMessage

# --- Setup ---
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

# Initialize Gemini LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    temperature=0.3,
    google_api_key=GOOGLE_API_KEY
)

# Initialize short-term memory (buffer of last 5 messages)
memory = ConversationBufferWindowMemory(k=5, return_messages=True)

# System prompt for structured agent behavior
SYSTEM_PROMPT = """
You are a specialized control chart monitoring agent in a manufacturing environment.
Your role is to observe new sensor readings and determine process stability using SPC (Statistical Process Control) logic.

Use this structured output:
@observe: Acknowledge the new reading.
@reason: Explain trend status or SPC rule violations, referencing the trend window.
@act: Recommend what action to take (e.g., monitor, alert supervisor, prep shutdown).
"""

# Sample input (you can swap this out for other sensor messages)
sensor_data = {
    "sensor": "vibration_motor_3A",
    "timestamp": "2025-06-18T11:47:09Z",
    "value": 0.89,
    "unit": "mm/s",
    "ucl": 0.90,
    "lcl": 0.50,
    "trend_window": [0.91, 0.94, 0.86, 0.89],
    "alert_enabled": True
}

# Prepare prompt
formatted_data = json.dumps(sensor_data, indent=2)
prompt = f"{SYSTEM_PROMPT}\n\nNew telemetry data:\n{formatted_data}"

# Run LLM with memory context
response = llm.invoke([HumanMessage(content=prompt)])

# Save exchange to memory
memory.save_context({"input": formatted_data}, {"output": response.content})

# Print the response
print("\n===== AGENT RESPONSE =====")
print(response.content)
